In [ ]:
###############################################################################################
re.findall(r'\n(.*?)\n', ligue_header)
arr = list(map(int, list_obj))
time.sleep(60)
season_comp_ids.insert(0, 'Friendly')

[f(x) if condition else g(x) for x in sequence]
[f(x) for x in sequence if condition] # without else

foo_des = foo.descendants # for eaach class elements in part soup 
for d in foo_des:
    if d.name == 'div' and d.get('class', 'opt r tb')

for foo in soup.find_all('div', attrs={'class': 'foo'}):
    bar = foo.find('div', attrs={'class': 'bar'})
    print(bar.text)

g = df.groupby('comp_id')['season_id'].agg(lambda x: x.unique().tolist())
###############################################################################################

In [1]:
import pickle
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime
from requests.exceptions import ConnectionError

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [2]:
# number of col_names in list must be equal number of rows df
def transform_columns_to_rows(df: pd.DataFrame, col_names: list):
    pd_series = pd.DataFrame()
    for i, col in zip(range(df.shape[0]), col_names):
        pd_series[col] = df.stack()[i]
    return pd_series 

In [3]:
def transform_date(list_obj: list):
    now = datetime.datetime.now()
    dates = []
    for obj in list_obj:
        if re.search(r'[a-zA-Z]', obj):
            date = ''.join(re.findall(r'[a-zA-Z]+', obj))
        elif (len(obj) == 12) & (re.search(r'[a-zA-Z]', obj) is None):
            date = obj.replace(',', '.' + str(now.year))
        else: 
            date = obj[:6] + str(now.year)[:2] + obj[6:8] + obj[9:15]
        dates.append(date)
    return dates

In [4]:
def double_slice_list(list_obj: list, septener: str):
    k = 2
    even = list_obj[k-1::k]
    odd = list_obj[k-2::k]
    all_res = []
    for x, y in zip(odd, even):
        res = x + septener + y
        all_res.append(res)
    return all_res

In [5]:
def cut_part_of_string(str_obj: str, start_board: str, end_board: str):
    reg_str =  start_board +'(.*?)' + end_board 
    return ''.join(re.findall(reg_str, str_obj))

In [6]:
def scrape_html(str_url: str): 
    html = requests.get(str_url).content
    soup = BeautifulSoup(html)
    
    all_matches_db = {}
    for each_tb in soup.find_all('div', {'class': 'live_comptt_bd'}):
        ligue_header = each_tb.find('div', {'class': 'block_header'}).get_text()
        ligue_header = ''.join(re.findall(r'\n (.*?)\n', ligue_header))  
        ligue_header =   'Friendly' if ligue_header == '' else ligue_header
        
        season_id = cut_part_of_string(str(each_tb),'season_id=', '\'')
        if (season_id == '') & (ligue_header != 'Friendly'):
            season_id = 'Cup' 
        elif ligue_header == 'Friendly': 
            season_id = 'Friendly'
        
        comp_id = each_tb.get('id')[3:] 
        
        game_ids = [x.get('dt-id') for x in each_tb.find_all('a', {'class': 'game_link'})]
        
        game_titles = [x.get('title') for x in each_tb.find_all('a', {'class': 'game_link'})]
        
        game_times_utc = [x.get_text() for x in each_tb.find_all('span', {'class': 'size10'})]
        game_times_utc = transform_date(game_times_utc)
        
        game_statuses = [x if re.search('[a-zA-Z]', str(x)) else 'Finished' for x in game_times_utc]
        
        game_times_utc = [x[:16] for x in game_times_utc if re.search(r'[a-zA-Z]?', str(x))]
        
        all_goals = [x.get_text() for x in each_tb.find_all('div', {'class': 'gls'})]
        all_goals = double_slice_list(all_goals, ':')

        stages = [x.get_text() for x in each_tb.find_all('div', {'class': 'stage'})]
        
        matches = dict()
        for game_id, game_utc, game_title, goals, game_status in zip(game_ids, game_times_utc, game_titles, all_goals, game_statuses): 
            matches.update({game_id: [ligue_header, comp_id, season_id, game_utc, game_title, goals, game_status]})
            
        all_matches_db.update(matches)
        
    return all_matches_db

In [7]:
# 7-11-2013 last day with bet's data on soccer365, 2697 - days before now
def create_date_list(numdays: int, start_year: int, start_month: int, start_day: int):
    base = datetime.date(start_year, start_month, start_day)
    date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
    return date_list

In [8]:
# function take url without date
def parsing_write_by_date(numdays: int, start_year: int, start_month: int, start_day: int, start_url='https://soccer365.me/online/&date='):
    date_list = create_date_list(numdays, start_year, start_month, start_day)
    all_matches_db = {}
    for date in date_list:
        print(date)
        main_url = start_url + str(date)
        matches_db = scrape_html(main_url)
        all_matches_db.update(matches_db)
        time.sleep(3)
    all_matches = open('all_matches', 'wb')
    pickle.dump(all_matches_db, all_matches)  
    all_matches.close()
    return print('Data saved')

In [9]:
# parsing_write_by_date(numdays=2700, start_year=2021, start_month=3, start_day=26)

In [9]:
with open('all_matches', 'rb') as f:
    all_matches = pickle.load(f)

In [10]:
df_to_transform = pd.DataFrame(all_matches)

In [11]:
df = transform_columns_to_rows(df_to_transform, ['ligue_header', 'comp_id', 'season_id', 'game_utc', 'game_title','goals', 'game_status'])

In [12]:
df.reset_index(inplace=True)
df.rename(columns={'index':'game_id'}, inplace=True)

In [13]:
df

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status
0,1458512,Segunda División,707,306,26.03.2021 20:00,Sporting Gijón - Alcorcón,0:0,Finished
1,1458506,Segunda División,707,306,26.03.2021 22:00,Castellon - Espanyol,1:3,Finished
2,1459331,First League,435,306,26.03.2021 14:00,Mykolaiv - Alians Lypova Dolyna,2:0,Finished
3,1459330,First League,435,306,26.03.2021 14:00,Polessya - Krystal Kherson,1:1,Finished
4,1459334,First League,435,306,26.03.2021 15:00,Kremin - Hirnyk-Sport,1:1,Finished
...,...,...,...,...,...,...,...,...
543580,10934224,Torneo Federal A,455,38,04.11.2013 00:30,Juventud Unida Universitario - Union Mar del Plata,2:0,Finished
543581,10968262,Division di Honor,463,38,04.11.2013 01:00,Nacional da Madeira - Dakota,1:3,Finished
543582,10925170,Primera D Metropolitana,460,38,04.11.2013 19:30,Claypole - Argentino Rosario,0:1,Finished
543583,10960778,Argentino B,458,38,04.11.2013 01:00,Huracan San Rafael - Pacífico,3:0,Finished


In [14]:
def find_game_events(game_ids: list, url='https://soccer365.me/games/', name_saved_file='events_games'):
    events_dict = {}
    for game_id, count in zip(game_ids, range(len(game_ids))):
        try:
            html = requests.get(url + game_id).content
            soup = BeautifulSoup(html)

            event_mins = [x.get_text() for x in soup.find_all('div', {'class':'event_min'})]
            event_mins = [cut_part_of_string(str(x), '^', '\'') for x in event_mins]
            event_hts = [x.find('div') for x in soup.find_all('div', {'class':'event_ht'})]
            event_hts = [cut_part_of_string(str(x), '=\"', '\">') for x in event_hts]
            event_ats = [x.find('div') for x in soup.find_all('div', {'class':'event_at'})]
            event_ats = [cut_part_of_string(str(x), '=\"', '\">') for x in event_ats]
            event_hts_ats = [x if x != '' else y for x, y in zip(event_hts, event_ats)]
            event_hts_ats = [cut_part_of_string(str(x), 'live_', '$') for x in event_hts_ats]

            stats_items = soup.find_all("div", {"class": "stats_item"})
            stats_dict = []
            for stats_item in stats_items:
                stats_title = stats_item.find("div", {"class": "stats_title"}).text
                stats_inf_1 = stats_item.find_all("div", {"class": "stats_inf"})[0].text
                stats_inf_2 = stats_item.find_all("div", {"class": "stats_inf"})[1].text
                stats_list = stats_title, stats_inf_1, stats_inf_2
                stats_dict.append(stats_list)

            try:
                preview_items = soup.find_all('div', {'class':'preview_item'})
                city_country = preview_items[0].find_all('span', {'class':'min_gray'})[0].text
            except (AttributeError, IndexError):   
                preview_items = city_country = ''

            try:
                prview_weath_tmp = preview_items[0].find('span', {'class':'prview_weath_tmp'}).text
                weather = cut_part_of_string(prview_weath_tmp, '^', '°')
                temp = cut_part_of_string(prview_weath_tmp, '\xa0\xa0', '$')
                weath_temp = weather, temp
            except (AttributeError, IndexError):
                prview_weath_tmp = weath_temp = ''

            try:
                viewers = cut_part_of_string(preview_items[1].text, ':', '$')
            except (AttributeError, IndexError):
                viewers = ''

            time.sleep(1.5)
            url_for_bets = url.replace('.me/', '.ru/')
            bets_html = requests.get(url_for_bets + game_id).content
            bets_soup = BeautifulSoup(bets_html)

            bet_titeles = [x.get_text() for x in bets_soup.find_all('div', {'class':'odds_coeff_title'})]

            try:
                coeffs = [x.get_text() for x in bets_soup.find_all('div', {'class':'odds_coeff'})[:len(bet_titeles)]]
            except (AttributeError, IndexError):
                coeffs = ''

            bet_coeffs = bet_titeles, coeffs

            events = {game_id:[event_mins, event_hts_ats, stats_dict, city_country, viewers, weath_temp, bet_coeffs]}
            events_dict.update(events)

            if (count % 500 == 0) & (count != 0) | (game_id == game_ids[-1]):
                print('Current_500_games_events_saved - {}'.format(game_id))

                events_games = open('pickle_files/' + name_saved_file + '_' + game_id, 'wb')
                pickle.dump(events_dict, events_games)  
                events_games.close()
                time.sleep(400) 

            time.sleep(1.5)    
        except ConnectionError:
            time.sleep(600) 
            find_game_events([game_id], name_saved_file=name_saved_file)

    return print('Data saved with last id: {}'.format(game_id))

In [16]:
# top_ligues_start_1 = ['12', '13', '14', '15', '16', '17', '18', '419', '420']
# top_ligues_start_2 = ['454', '456', '474', '483', '485', '550', '554', '560', '565']

In [17]:
# top_ligues_end =  ['577', '581', '587', '591','596', '601', '677', '681'] 
# top_ligues_end_2 = ['684', '695', '699', '707', '712', '716', '723', '727'] 

In [15]:
# second_ligues_start = [ '419', '424', '435', '436', '437', '444', '446', '449', '450', '457', '459', '464', '473', '477', '491']
# second_ligues_start_2  = ['493', '496', '498', '500', '502', '504', '507', '512', '516', '518', '523', '532', '539', '540', '542']
# second_ligues_middle = [ '544', '545', '546', '548', '551', '555', '562', '567', '576', '585', '586', '589', '595', '599', '602']
# second_ligues_middle_2 =  ['606', '621', '622', '625', '626', '628', '632', '634', '636', '637', '642', '646', '647', '648', '653']
# second_ligues_end = ['654', '655', '657', '659', '660', '667', '672', '674', '675', '676', '679', '685', '687', '691', '692', '697']
second_ligues_end_2 = ['700', '703', '704', '714', '715', '720', '721', '725', '727', '733''735', '747', '761', '912', '1157', '1259'] 

In [16]:
leagues = second_ligues_end_2
leagues_name = 'second_ligues_end_2'

In [17]:
game_ids = df.game_id[df.comp_id.isin(leagues)].to_list()
len(game_ids)

16915

In [43]:
# game_ids.index('10934906') 

In [44]:
# Start it sell
# find_game_events(game_ids[16505:], name_saved_file=leagues_name)

In [39]:
with open('pickle_files/second_ligues_end_2_10934879', 'rb') as f:
    events_games = pickle.load(f)

In [40]:
df_pickle = pd.DataFrame(events_games)

In [29]:
# First game_id
# 11967929 - 12997 - second_ligues_start_11967929
# 11925466 - 5498  - second_ligues_start_10991305
# 11258622 - 2000 - second_ligues_start__11028911
# 10856567 - 3200 - second_ligues_start_10924530
# 1507965 - 5500 - second_ligues_start_2_11935838
# 11935833 - 4840 - second_ligues_start_2_10932840

# 1445410 - 9500 - second_ligues_middle_12606526
# 12606530 - 5500 - second_ligues_middle_11937265
# 11937267 - 11630 - second_ligues_middle_10961666
# 1521058 - 10000 - second_ligues_middle_2_12621697
# 12621701 - 6500 - second_ligues_middle_2_11478148
# 11511623 - 5500- second_ligues_middle_2_14513946
# 11065258 - 2000 - second_ligues_middle_2_14385660
# 10884541 - 650 - second_ligues_middle_2_10884423

# 1427554 - 6000 - second_ligues_end_14128217
# 14128214 - 500 - second_ligues_end_14145950
# 14145944 - 5000 - second_ligues_end_11902890
# 11914011 - 9000 - second_ligues_end_10869236
# 11012263 - 2500 - second_ligues_end_10915569

# 1533922 - 8000 - second_ligues_end_2_11907597
# 511894218 - 500 - second_ligues_end_2_12558586
# 12558685 - 6500 - second_ligues_end_2_10861138
# 11057180 - 1000 - second_ligues_end_2_10932349
# 10932346 - 500 - second_ligues_end_2_10934906
# 10934905 - 409 - second_ligues_end_2_10625206

# 1432501 - 7500 - top_ligues_end_12563304
# 12563302 - 9130 - top_ligues_end_10911255
# 12567313 - 9146 - top_ligues_end_2_10909453
# 1458512 - 7500 - top_ligues_end_2_12567311
# 1529443  - 19084 - top_ligues_start_2_1090378
# 11920204 - 9495 - top_ligues_start_1_10900138
# 1442836  - 2001 - top_ligues_start_1_14893395
# 14893402 - 501  - top_ligues_start_1_14919434
# 14919433 - 501  - top_ligues_start_1_14900239
# 14900237 - 1001 - top_ligues_start_1_14101959
# 14898253 - 1001 - top_ligues_start_1_14898150
# 14898157 - 7995 - top_ligues_start_1_11920200

In [42]:
# df_pickle

In [36]:
df_pickle = transform_columns_to_rows(df_pickle, ['event_mins', 'event_hts_ats', 'stats_dict', 'city_country', 'viewers', 'weath_temp'])

In [37]:
df_pickle.reset_index(inplace=True)
df_pickle.rename(columns={'index':'game_id'}, inplace=True)

In [38]:
df_pickle.shape

(409, 7)

In [41]:
df_pickle.tail()

,game_id,event_mins,event_hts_ats,stats_dict,city_country,viewers,weath_temp
2486,14411344,[],[],[],"Craiova, Romania",,"(+12, overcast)"
2487,10952183,"[18, 27, 38, 81]","[goal, goal, goal, pengoal]",[],Romania,,
2488,14411345,[],[],[],Romania,,
2489,10915568,"[25, 32, 72, 88]","[yellowcard, yellowcard, yellowcard, pengoal]",[],"Jaroslavl, Russia",,"(+6, partly cloudy)"
2490,10915569,"[25, 50, 57, 62, 77, 90, 90]","[yellowcard, goal, yellowcard, yellowcard, yellowcard, yellowcard, yellowcard]",[],"Dzerzhinsk, Russia",,"(+2, light drizzle)"


################################################################################################################################

In [ ]:
# For get tables of competitions 
# https://soccer365.ru/competitions/16/ - current tables
# https://soccer365.ru/competitions/16/2017-2018/ - for seasons

In [ ]:
# html = requests.get('https://soccer365.me/?c=live&a=showtable&competition_id=483&season_id=307').content
# <tr class="adv_kef_wgt_odd">

In [16]:
# replace 'Finished' not 'Postponed' games wih '-:-' goals for 'Canseled'
df[df.game_id == '1510009']

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status
151,1510009,Campeonato de Portugal,680,306,03.01.2021 13:00,Pevidem - Camacha,-:-,Finished
